In [ ]:
from reportlab.lib.units import inch
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, PageTemplate, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import Paragraph, Frame, PageBreak, Spacer, BaseDocTemplate
from datetime import datetime

class MyDocTemplate(BaseDocTemplate):
    def __init__(self, filename, **kwargs):
        super().__init__(filename, **kwargs)
        
        # Defina o cabeçalho e o rodapé
        self.header = Image("caminho/para/imagem_esquerda.png", width=1.5*inch, height=1*inch)
        self.footer = ParagraphStyle("footer", parent=getSampleStyleSheet()["Normal"])
        
        self.addPageTemplates([
            PageTemplate(id="header", frames=[Frame(0, 0, 7*inch, 10*inch, id="header_frame", showBoundary=0)]),
            PageTemplate(id="footer", frames=[Frame(0, 0, 7*inch, 0.5*inch, id="footer_frame", showBoundary=0)]),
        ])
    
    def build(self, flowables, onFirstPage=None, onLaterPages=None):
        # Define a função que será chamada na primeira página
        def first_page(canvas, document):
            canvas.saveState()
            canvas.drawImage("caminho/para/imagem_meio.png", x=3.5*inch, y=0.25*inch, width=1.5*inch, height=0.5*inch)
            if onFirstPage:
                onFirstPage(canvas, document)
            canvas.restoreState()
        
        # Define a função que será chamada nas páginas subsequentes
        def later_pages(canvas, document):
            canvas.saveState()
            canvas.setFont("Helvetica", 8)
            canvas.drawString(0.25*inch, 0.25*inch, datetime.now().strftime("%d/%m/%Y %H:%M"))
            canvas.drawRightString(6.75*inch, 0.25*inch, "Página %d de %d" % (canvas.getPageNumber(), self.page))
            if onLaterPages:
                onLaterPages(canvas, document)
            canvas.restoreState()
        
        self.onFirstPage = first_page
        self.onLaterPages = later_pages
        super().build(flowables)

# Crie o documento
doc = MyDocTemplate("caminho/para/arquivo.pdf", pagesize=letter)

# Adicione o conteúdo ao documento
content = []
# Adicione suas imagens, parágrafos, etc., ao conteúdo

doc.build(content)

In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.platypus import Paragraph
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

from datetime import datetime

class Create_PDF():
    def __init__(self, nome_arquivo="relatorio_inventario.pdf") -> None:
        # self.nome_arquivo = nome_arquivo
        self._c = canvas.Canvas(nome_arquivo, pagesize=A4)
        # Obter a largura da página
        self._largura_pagina, self._altura_pagina = A4
        
    def get_nome_arquivo(self): return self._c._filename

    def centralizar_texto(self, titulo): 
        # Para saber o compriumento do texto
        # # largura_texto = self.c.stringWidth(titulo, "Helvetica-Bold", 16)
        # # x_titulo = (self._largura_pagina - largura_texto) / 2
        
        # Calcular a posição X para centralizar o título
        x_titulo = (self._largura_pagina) / 2

        # Definir a posição Y para o título
        y_titulo = self._altura_pagina - 30  # Você pode ajustar a posição vertical conforme necessário

        # Configurar o título do relatório
        self._c.setFont("Helvetica-Bold", 16)

        # Desenhar o título centralizado
        self._c.drawCentredString(x_titulo, y_titulo, titulo)

        self._altura_pagina = y_titulo - 40  # Iniciar logo abaixo do título

    def gerar_relatorio_inventario(self, dados_inventario):
        # Configurar os dados do inventário
        self._c.setFont("Helvetica", 12)
        y = self._altura_pagina  # Posição vertical inicial para os dados
        for item in dados_inventario:
            self._c.drawString(100, y, f"Item: {item['nome']}")
            self._c.drawString(300, y, f"Quantidade: {item['quantidade']}")
            y -= 20  # Mover para a próxima linha

    def salvar_pdf(self): self._c.save()

    def inserir_imagem(self, imagem_path, ):
        # Inserir a imagem
        imagem_x = 400  # Posição X da imagem
        imagem_y = self._altura_pagina - 150  # Posição Y da imagem
        # largura_imagem = 100  # Largura da imagem
        # altura_imagem = 100  # Altura da imagem

        self._c.drawImage(imagem_path, x=imagem_x, y=imagem_y, width=200, mask='auto', preserveAspectRatio=True)
        # self._c.drawImage(imagem_path, x=imagem_x, y=imagem_y, mask='auto')

    def cabecalho(self):
        self._c.drawImage(image='IMAGE/cabeçalho esq.png', x=15, y=self._altura_pagina - 195, width=180, mask='auto', preserveAspectRatio=True)
        self._c.drawImage(image='IMAGE/cabeçalho meio.png', x=225, y=self._altura_pagina - 95, width=140, mask='auto', preserveAspectRatio=True)

        # Criar um estilo personalizado com a fonte desejada
        estilo_personalizado = ParagraphStyle(
            "estilo_personalizado",
            parent=getSampleStyleSheet()["Heading4"],  # Copie as configurações de estilo padrão
            fontName="Courier-Bold",  # Substitua pelo nome da fonte desejada
            fontSize=12
        )

        p = Paragraph('Rua: Gabriel Gonçalves nº 351 – Aleixo - CEP 69060–010\nTel.: 3642-3064 - Manaus-AM\nwww.manaus.am.gov.br', estilo_personalizado)
        p.wrapOn(self._c, 200, 100)
        p.drawOn(self._c, 380, self._altura_pagina - 75)

        # self.centralizar_texto("INVENTÁRIO DE BENS IMÓVEIS")
        # self.gerar_relatorio_inventario(inventario)
        print(f"Cabeçalho do Relatório de inventário gerado em {self.get_nome_arquivo()}")

    def rodape(self):
        # Defina o estilo do texto do rodapé
        estilo_rodape = ParagraphStyle(
            'rodape',
            parent=getSampleStyleSheet()['Normal'],
            fontSize=10,
            alignment=1
        )

        # Obtenha a data atual
        data_atual = datetime.now().strftime('%d/%m/%Y - %H:%M:%S')

        # Obtenha o número da página atual
        numero_pagina = self._c.getPageNumber()

        # Defina o texto do rodapé
        texto_rodape = f'{data_atual}  |  Página {numero_pagina}'

        # Crie um objeto Paragraph com o texto do rodapé e o estilo definido
        paragrafo_rodape = Paragraph(texto_rodape, estilo_rodape)

        # Desenhe o parágrafo do rodapé no self._c
        paragrafo_rodape.wrap(self._altura_pagina, 20)
        paragrafo_rodape.drawOn(self._c, 0, 30)

# if __name__ == "__main__":
inventario = [
    {"nome": "Produto A", "quantidade": 10},
    {"nome": "Produto B", "quantidade": 20},
    {"nome": "Produto C", "quantidade": 15},
    {"nome": "Produto D", "quantidade": 25},
    {"nome": "Produto E", "quantidade": 20},
    {"nome": "Produto F", "quantidade": 30},
    {"nome": "Produto G", "quantidade": 25},
    {"nome": "Produto H", "quantidade": 35},
]

pdf = Create_PDF()
pdf.cabecalho()
pdf.rodape()
pdf.salvar_pdf()


Cabeçalho do Relatório de inventário gerado em relatorio_inventario.pdf


In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.platypus import Paragraph
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

class Create_PDF():
    def __init__(self, nome_arquivo="relatorio_inventario.pdf") -> None:
        # self.nome_arquivo = nome_arquivo
        self._c = canvas.Canvas(nome_arquivo, pagesize=A4)
        # Obter a largura da página
        self._largura_pagina, self._altura_pagina = A4
        
    def get_nome_arquivo(self): return self._c._filename

    def centralizar_texto(self, titulo): 
        # Para saber o compriumento do texto
        # # largura_texto = self.c.stringWidth(titulo, "Helvetica-Bold", 16)
        # # x_titulo = (self._largura_pagina - largura_texto) / 2
        
        # Calcular a posição X para centralizar o título
        x_titulo = (self._largura_pagina) / 2

        # Definir a posição Y para o título
        y_titulo = self._altura_pagina - 30  # Você pode ajustar a posição vertical conforme necessário

        # Configurar o título do relatório
        self._c.setFont("Helvetica-Bold", 16)

        # Desenhar o título centralizado
        self._c.drawCentredString(x_titulo, y_titulo, titulo)

        self._altura_pagina = y_titulo - 40  # Iniciar logo abaixo do título

    def gerar_relatorio_inventario(self, dados_inventario):
        # Configurar os dados do inventário
        self._c.setFont("Helvetica", 12)
        y = self._altura_pagina  # Posição vertical inicial para os dados
        for item in dados_inventario:
            self._c.drawString(100, y, f"Item: {item['nome']}")
            self._c.drawString(300, y, f"Quantidade: {item['quantidade']}")
            y -= 20  # Mover para a próxima linha

    def salvar_pdf(self): self._c.save()

    def inserir_imagem(self, imagem_path, ):
        # Inserir a imagem
        imagem_x = 400  # Posição X da imagem
        imagem_y = self._altura_pagina - 150  # Posição Y da imagem
        # largura_imagem = 100  # Largura da imagem
        # altura_imagem = 100  # Altura da imagem

        self._c.drawImage(imagem_path, x=imagem_x, y=imagem_y, width=200, mask='auto', preserveAspectRatio=True)
        # self._c.drawImage(imagem_path, x=imagem_x, y=imagem_y, mask='auto')

    def cabecalho(self):
        self._c.drawImage(image='IMAGE/cabeçalho esq.png', x=15, y=self._altura_pagina - 195, width=180, mask='auto', preserveAspectRatio=True)
        self._c.drawImage(image='IMAGE/cabeçalho meio.png', x=225, y=self._altura_pagina - 95, width=140, mask='auto', preserveAspectRatio=True)

        # Criar um estilo personalizado com a fonte desejada
        estilo_personalizado = ParagraphStyle(
            "estilo_personalizado",
            parent=getSampleStyleSheet()["Heading4"],  # Copie as configurações de estilo padrão
            fontName="Courier-Bold",  # Substitua pelo nome da fonte desejada
            fontSize=12
        )

        p = Paragraph('Rua: Gabriel Gonçalves nº 351 – Aleixo - CEP 69060–010\nTel.: 3642-3064 - Manaus-AM\nwww.manaus.am.gov.br', estilo_personalizado)
        p.wrapOn(self._c, 200, 100)
        p.drawOn(self._c, 380, self._altura_pagina - 75)

        # self.centralizar_texto("INVENTÁRIO DE BENS IMÓVEIS")
        # self.gerar_relatorio_inventario(inventario)
        print(f"Cabeçalho do Relatório de inventário gerado em {self.get_nome_arquivo()}")

# if __name__ == "__main__":
inventario = [
    {"nome": "Produto A", "quantidade": 10},
    {"nome": "Produto B", "quantidade": 20},
    {"nome": "Produto C", "quantidade": 15},
    {"nome": "Produto D", "quantidade": 25},
    {"nome": "Produto E", "quantidade": 20},
    {"nome": "Produto F", "quantidade": 30},
    {"nome": "Produto G", "quantidade": 25},
    {"nome": "Produto H", "quantidade": 35},
]

pdf = Create_PDF()
pdf.cabecalho()
pdf.salvar_pdf()


Cabeçalho do Relatório de inventário gerado em relatorio_inventario.pdf


In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.platypus import Paragraph
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

class Create_PDF():
    def __init__(self, nome_arquivo="relatorio_inventario.pdf") -> None:
        # self.nome_arquivo = nome_arquivo
        self._c = canvas.Canvas(nome_arquivo, pagesize=A4)
        # Obter a largura da página
        self._largura_pagina, self._altura_pagina = A4
        
    def get_nome_arquivo(self): return self._c._filename

    def centralizar_texto(self, titulo): 
        # Para saber o compriumento do texto
        # # largura_texto = self.c.stringWidth(titulo, "Helvetica-Bold", 16)
        # # x_titulo = (self._largura_pagina - largura_texto) / 2
        
        # Calcular a posição X para centralizar o título
        x_titulo = (self._largura_pagina) / 2

        # Definir a posição Y para o título
        y_titulo = self._altura_pagina - 30  # Você pode ajustar a posição vertical conforme necessário

        # Configurar o título do relatório
        self._c.setFont("Helvetica-Bold", 16)

        # Desenhar o título centralizado
        self._c.drawCentredString(x_titulo, y_titulo, titulo)

        self._altura_pagina = y_titulo - 40  # Iniciar logo abaixo do título

    def gerar_relatorio_inventario(self, dados_inventario):
        # Configurar os dados do inventário
        self._c.setFont("Helvetica", 12)
        y = self._altura_pagina  # Posição vertical inicial para os dados
        for item in dados_inventario:
            self._c.drawString(100, y, f"Item: {item['nome']}")
            self._c.drawString(300, y, f"Quantidade: {item['quantidade']}")
            y -= 20  # Mover para a próxima linha

    def salvar_pdf(self): self._c.save()

    def inserir_imagem(self, imagem_path, ):
        # Inserir a imagem
        imagem_x = 400  # Posição X da imagem
        imagem_y = self._altura_pagina - 150  # Posição Y da imagem
        # largura_imagem = 100  # Largura da imagem
        # altura_imagem = 100  # Altura da imagem

        self._c.drawImage(imagem_path, x=imagem_x, y=imagem_y, width=200, mask='auto', preserveAspectRatio=True)
        # self._c.drawImage(imagem_path, x=imagem_x, y=imagem_y, mask='auto')

    def cabecalho(self):
        self._c.drawImage(image='IMAGE/cabeçalho esq.png', x=15, y=self._altura_pagina - 195, width=180, mask='auto', preserveAspectRatio=True)
        self._c.drawImage(image='IMAGE/cabeçalho meio.png', x=225, y=self._altura_pagina - 95, width=140, mask='auto', preserveAspectRatio=True)

        # Criar um estilo personalizado com a fonte desejada
        estilo_personalizado = ParagraphStyle(
            "estilo_personalizado",
            parent=getSampleStyleSheet()["Heading4"],  # Copie as configurações de estilo padrão
            fontName="Courier-Bold",  # Substitua pelo nome da fonte desejada
            fontSize=12
        )

        p = Paragraph('Rua: Gabriel Gonçalves nº 351 – Aleixo - CEP 69060–010\nTel.: 3642-3064 - Manaus-AM\nwww.manaus.am.gov.br', estilo_personalizado)
        p.wrapOn(self._c, 200, 100)
        p.drawOn(self._c, 380, self._altura_pagina - 75)

        # self.centralizar_texto("INVENTÁRIO DE BENS IMÓVEIS")
        # self.gerar_relatorio_inventario(inventario)
        print(f"Cabeçalho do Relatório de inventário gerado em {self.get_nome_arquivo()}")


# if __name__ == "__main__":
inventario = [
    {"nome": "Produto A", "quantidade": 10},
    {"nome": "Produto B", "quantidade": 20},
    {"nome": "Produto C", "quantidade": 15},
    {"nome": "Produto D", "quantidade": 25},
    {"nome": "Produto E", "quantidade": 20},
    {"nome": "Produto F", "quantidade": 30},
    {"nome": "Produto G", "quantidade": 25},
    {"nome": "Produto H", "quantidade": 35},
]

pdf = Create_PDF()
pdf.cabecalho()
pdf.salvar_pdf()


Cabeçalho do Relatório de inventário gerado em relatorio_inventario.pdf


In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.platypus import Paragraph
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

class Create_PDF():
    def __init__(self, nome_arquivo="relatorio_inventario.pdf") -> None:
        # self.nome_arquivo = nome_arquivo
        self._c = canvas.Canvas(nome_arquivo, pagesize=A4)
        # Obter a largura da página
        self._largura_pagina, self._altura_pagina = A4
        
    def get_nome_arquivo(self): return self._c._filename

    def centralizar_texto(self, titulo): 
        # Para saber o compriumento do texto
        # # largura_texto = self.c.stringWidth(titulo, "Helvetica-Bold", 16)
        # # x_titulo = (self._largura_pagina - largura_texto) / 2
        
        # Calcular a posição X para centralizar o título
        x_titulo = (self._largura_pagina) / 2

        # Definir a posição Y para o título
        y_titulo = self._altura_pagina - 30  # Você pode ajustar a posição vertical conforme necessário

        # Configurar o título do relatório
        self._c.setFont("Helvetica-Bold", 16)

        # Desenhar o título centralizado
        self._c.drawCentredString(x_titulo, y_titulo, titulo)

        self._altura_pagina = y_titulo - 40  # Iniciar logo abaixo do título

    def gerar_relatorio_inventario(self, dados_inventario):
        # Configurar os dados do inventário
        self._c.setFont("Helvetica", 12)
        y = self._altura_pagina  # Posição vertical inicial para os dados
        for item in dados_inventario:
            self._c.drawString(100, y, f"Item: {item['nome']}")
            self._c.drawString(300, y, f"Quantidade: {item['quantidade']}")
            y -= 20  # Mover para a próxima linha

    def salvar_pdf(self): self._c.save()

    def inserir_imagem(self, imagem_path, ):
        # Inserir a imagem
        imagem_x = 400  # Posição X da imagem
        imagem_y = self._altura_pagina - 150  # Posição Y da imagem
        # largura_imagem = 100  # Largura da imagem
        # altura_imagem = 100  # Altura da imagem

        self._c.drawImage(imagem_path, x=imagem_x, y=imagem_y, width=200, mask='auto', preserveAspectRatio=True)
        # self._c.drawImage(imagem_path, x=imagem_x, y=imagem_y, mask='auto')

    def cabecalho(self):
        self._c.drawImage(image='IMAGE/cabeçalho esq.png', x=15, y=self._altura_pagina - 195, width=180, mask='auto', preserveAspectRatio=True)
        self._c.drawImage(image='IMAGE/cabeçalho meio.png', x=225, y=self._altura_pagina - 95, width=140, mask='auto', preserveAspectRatio=True)

        # Criar um estilo personalizado com a fonte desejada
        estilo_personalizado = ParagraphStyle(
            "estilo_personalizado",
            parent=getSampleStyleSheet()["Heading4"],  # Copie as configurações de estilo padrão
            fontName="Courier-Bold",  # Substitua pelo nome da fonte desejada
            fontSize=12
        )

        p = Paragraph('Rua: Gabriel Gonçalves nº 351 – Aleixo - CEP 69060–010\nTel.: 3642-3064 - Manaus-AM\nwww.manaus.am.gov.br', estilo_personalizado)
        p.wrapOn(self._c, 200, 100)
        p.drawOn(self._c, 380, self._altura_pagina - 75)

        # self.centralizar_texto("INVENTÁRIO DE BENS IMÓVEIS")
        # self.gerar_relatorio_inventario(inventario)
        print(f"Cabeçalho do Relatório de inventário gerado em {self.get_nome_arquivo()}")


# if __name__ == "__main__":
inventario = [
    {"nome": "Produto A", "quantidade": 10},
    {"nome": "Produto B", "quantidade": 20},
    {"nome": "Produto C", "quantidade": 15},
    {"nome": "Produto D", "quantidade": 25},
    {"nome": "Produto E", "quantidade": 20},
    {"nome": "Produto F", "quantidade": 30},
    {"nome": "Produto G", "quantidade": 25},
    {"nome": "Produto H", "quantidade": 35},
]

pdf = Create_PDF()
pdf.cabecalho()
pdf.salvar_pdf()


Cabeçalho do Relatório de inventário gerado em relatorio_inventario.pdf


In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.platypus import Paragraph
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

class Create_PDF():
    def __init__(self, nome_arquivo="relatorio_inventario.pdf") -> None:
        # self.nome_arquivo = nome_arquivo
        self._c = canvas.Canvas(nome_arquivo, pagesize=A4)
        # Obter a largura da página
        self._largura_pagina, self._altura_pagina = A4
        
    def get_nome_arquivo(self): return self._c._filename

    def centralizar_texto(self, titulo): 
        # Para saber o compriumento do texto
        # # largura_texto = self.c.stringWidth(titulo, "Helvetica-Bold", 16)
        # # x_titulo = (self._largura_pagina - largura_texto) / 2
        
        # Calcular a posição X para centralizar o título
        x_titulo = (self._largura_pagina) / 2

        # Definir a posição Y para o título
        y_titulo = self._altura_pagina - 30  # Você pode ajustar a posição vertical conforme necessário

        # Configurar o título do relatório
        self._c.setFont("Helvetica-Bold", 16)

        # Desenhar o título centralizado
        self._c.drawCentredString(x_titulo, y_titulo, titulo)

        self._altura_pagina = y_titulo - 40  # Iniciar logo abaixo do título

    def gerar_relatorio_inventario(self, dados_inventario):
        # Configurar os dados do inventário
        self._c.setFont("Helvetica", 12)
        y = self._altura_pagina  # Posição vertical inicial para os dados
        for item in dados_inventario:
            self._c.drawString(100, y, f"Item: {item['nome']}")
            self._c.drawString(300, y, f"Quantidade: {item['quantidade']}")
            y -= 20  # Mover para a próxima linha

    def salvar_pdf(self): self._c.save()

    def inserir_imagem(self, imagem_path, ):
        # Inserir a imagem
        imagem_x = 400  # Posição X da imagem
        imagem_y = self._altura_pagina - 150  # Posição Y da imagem
        # largura_imagem = 100  # Largura da imagem
        # altura_imagem = 100  # Altura da imagem

        self._c.drawImage(imagem_path, x=imagem_x, y=imagem_y, width=200, mask='auto', preserveAspectRatio=True)
        # self._c.drawImage(imagem_path, x=imagem_x, y=imagem_y, mask='auto')

    def cabecalho(self):
        self._c.drawImage(image='IMAGE/cabeçalho esq.png', x=15, y=self._altura_pagina - 195, width=180, mask='auto', preserveAspectRatio=True)
        self._c.drawImage(image='IMAGE/cabeçalho meio.png', x=225, y=self._altura_pagina - 95, width=140, mask='auto', preserveAspectRatio=True)

        # Criar um estilo personalizado com a fonte desejada
        estilo_personalizado = ParagraphStyle(
            "estilo_personalizado",
            parent=getSampleStyleSheet()["Heading4"],  # Copie as configurações de estilo padrão
            fontName="Courier-Bold",  # Substitua pelo nome da fonte desejada
            fontSize=12
        )

        p = Paragraph('Rua: Gabriel Gonçalves nº 351 – Aleixo - CEP 69060–010\nTel.: 3642-3064 - Manaus-AM\nwww.manaus.am.gov.br', estilo_personalizado)
        p.wrapOn(self._c, 200, 100)
        p.drawOn(self._c, 380, self._altura_pagina - 75)

        # self.centralizar_texto("INVENTÁRIO DE BENS IMÓVEIS")
        # self.gerar_relatorio_inventario(inventario)
        print(f"Cabeçalho do Relatório de inventário gerado em {self.get_nome_arquivo()}")


# if __name__ == "__main__":
inventario = [
    {"nome": "Produto A", "quantidade": 10},
    {"nome": "Produto B", "quantidade": 20},
    {"nome": "Produto C", "quantidade": 15},
    {"nome": "Produto D", "quantidade": 25},
    {"nome": "Produto E", "quantidade": 20},
    {"nome": "Produto F", "quantidade": 30},
    {"nome": "Produto G", "quantidade": 25},
    {"nome": "Produto H", "quantidade": 35},
]

pdf = Create_PDF()
pdf.cabecalho()
pdf.salvar_pdf()


Cabeçalho do Relatório de inventário gerado em relatorio_inventario.pdf


In [ ]:
for font in pdf._c.getAvailableFonts():
    print(font)

Courier
Courier-Bold
Courier-BoldOblique
Courier-Oblique
Helvetica
Helvetica-Bold
Helvetica-BoldOblique
Helvetica-Oblique
Symbol
Times-Bold
Times-BoldItalic
Times-Italic
Times-Roman
ZapfDingbats


In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

def quebrar_texto(texto, largura_maxima, estilo):
    # Crie um objeto Paragraph com o texto e estilo
    p = Paragraph(texto, estilo)

    # Obtenha a largura do texto
    largura_texto, _ = p.wrap(largura_maxima, 10)  # 1000 é uma altura de espaço em branco suficientemente grande

    # Verifique se o texto cabe na linha atual
    if largura_texto <= largura_maxima:
        return [p]  # O texto cabe em uma linha

    # Divida o texto em várias linhas
    linhas = []
    while len(texto) > 0:
        linha, texto = texto[:10], texto[10:]  # Divida o texto em segmentos menores
        p = Paragraph(linha, estilo)
        linhas.append(p)

    return linhas

def criar_documento_com_quebra_de_linha(nome_arquivo, texto):
    doc = SimpleDocTemplate(nome_arquivo, pagesize=letter)
    story = []

    # Defina um estilo para o texto
    # estilo = getSampleStyleSheet()['Title']
    estilo = getSampleStyleSheet()['Heading1']
    # estilo = getSampleStyleSheet()['Heading2']

    # Largura máxima da linha (ajuste conforme necessário)
    largura_maxima = 20

    # Quebre o texto em várias linhas
    linhas = quebrar_texto(texto, largura_maxima, estilo)

    # Adicione as linhas à história do documento
    for linha in linhas:
        story.append(linha)

    doc.build(story)

if __name__ == "__main__":
    texto = "Este é um exemplo de texto longo que será quebrado automaticamente para caber na largura da página.\nEste é um exemplo de texto longo que será quebrado automaticamente para caber na largura da página.\nEste é um exemplo de texto longo que será quebrado automaticamente para caber na largura da página."

    nome_do_arquivo = "documento_com_quebra_de_linha.pdf"

    criar_documento_com_quebra_de_linha(nome_do_arquivo, texto)
    print(f"Documento com quebra de linha gerado em {nome_do_arquivo}")


Documento com quebra de linha gerado em documento_com_quebra_de_linha.pdf
